In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from sklearn.metrics import mean_squared_error

import tensorflow as tf

tf.python.control_flow_ops = tf

 

Using TensorFlow backend.


In [2]:
# K.set_image_dim_ordering('tf')
K.set_image_dim_ordering('th')

In [3]:
K.image_dim_ordering()

'th'

In [4]:
batch_size = 32

In [5]:
x_tr, y_tr = np.load('x_tr.npy'), np.load('y_tr.npy')
x_te, y_te = np.load('x_te.npy'), np.load('y_te.npy')

In [6]:
y_tr = y_tr / 90.
y_te = y_te / 90.

In [7]:
img_rows, img_cols = 28, 28

In [8]:
x_tr = x_tr.reshape(x_tr.shape[0], 1, img_rows, img_cols)
x_te = x_te.reshape(x_te.shape[0], 1, img_rows, img_cols)
input_shape = (1, img_rows, img_cols)

In [113]:
nb_filters = 32
# size of pooling area for max pooling
pool_size = (4, 4)
# convolution kernel size
kernel_size = (3, 3)

In [114]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# model.add(Dense(32))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
model.add(Dense(1))
# model.add(Activation('sigmoid'))

In [115]:
from keras.optimizers import Adadelta

In [116]:
model.compile(loss='mse',
              optimizer=Adadelta(lr=1.),
              metrics=[])

In [117]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_11 (Convolution2D) (None, 32, 26, 26)    320         convolution2d_input_6[0][0]      
____________________________________________________________________________________________________
activation_19 (Activation)       (None, 32, 26, 26)    0           convolution2d_11[0][0]           
____________________________________________________________________________________________________
convolution2d_12 (Convolution2D) (None, 32, 24, 24)    9248        activation_19[0][0]              
____________________________________________________________________________________________________
activation_20 (Activation)       (None, 32, 24, 24)    0           convolution2d_12[0][0]           
___________________________________________________________________________________________

In [118]:
model.fit(x_tr, y_tr, batch_size=64, nb_epoch=10,
          verbose=1, validation_data=(x_te, y_te))

Train on 12000 samples, validate on 1000 samples
Epoch 1/10
12000/12000 [==============================] - 44s - loss: 0.1547 - val_loss: 0.0368
Epoch 2/10
12000/12000 [==============================] - 58s - loss: 0.0559 - val_loss: 0.0447
Epoch 3/10
12000/12000 [==============================] - 61s - loss: 0.0455 - val_loss: 0.0352
Epoch 4/10
12000/12000 [==============================] - 55s - loss: 0.0398 - val_loss: 0.0312
Epoch 5/10
12000/12000 [==============================] - 54s - loss: 0.0368 - val_loss: 0.0287
Epoch 6/10
12000/12000 [==============================] - 54s - loss: 0.0346 - val_loss: 0.0256
Epoch 7/10
12000/12000 [==============================] - 56s - loss: 0.0324 - val_loss: 0.0270
Epoch 8/10
12000/12000 [==============================] - 55s - loss: 0.0298 - val_loss: 0.0234
Epoch 9/10
12000/12000 [==============================] - 53s - loss: 0.0290 - val_loss: 0.0212
Epoch 10/10
12000/12000 [==============================] - 57s - loss: 0.0280 - val_los

In [119]:
preds = model.predict(x_te)

In [120]:
preds_tr = model.predict(x_tr)

In [121]:
mean_squared_error(y_tr*90, preds_tr*90) 

72.916813948447057

In [122]:
mean_squared_error(y_te*90, preds*90)

184.2598822624266

In [123]:
mean_squared_error(y_te, preds)

0.022748133612645261

In [124]:
from matplotlib import pyplot as plt

In [ ]:
plt.hist(np.abs(preds - y_te), bins=10)
plt.show()